In [4]:
%load_ext ipython_pytest

The ipython_pytest extension is already loaded. To reload it, use:
  %reload_ext ipython_pytest


In [5]:
'''Dask Cluster'''
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:61899 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 7.45 GB


In [10]:
%%pytest

import xarray as xr
import pytest

from mean_sst import ParameterTypeError, BboxLengthError, BboxCellsizeError, LongitudeValueError, LatitudeValueError, TimeframeLengthError, TimeframeValueError  
from mean_sst import createSubset, wrapper_mean_sst, exceptions_mean_sst, mean_sst

ds = xr.open_dataset("C:/Users/Digitainer/Documents/Uni_Muenster/WiSe_2020/Geosoftware2/Jupyter/sst.day.mean.1984-03-4days.nc", chunks={"time": "auto"})

'''test function createSubset'''

def test_createSubsetNormalCase():
    x = createSubset(ds, 0, -20, 100, 20)
    for i in x["lon"].values:
        assert(i >= 0 and i <= 100)
    for i in x["lat"].values:
        assert(i >= -20 and i <= 20)
        
def test_createSubsetMinLonGreaterThanMaxLonCase():
    x = createSubset(ds, 300, -20, 50, 20)
    assert(x["lon"].values[0] >= 300)
    assert(x["lon"].values[-1] <= 50)
    assert(x["lat"].values[0] >= -20)
    assert(x["lat"].values[-1] <= 20)
        
'''test function wrapper_mean_sst'''
    
def test_wrapper_mean_sst():
    x = wrapper_mean_sst(ds, ['1984-03-01', '1984-03-04'], [0,-20,70,20])
    assert(x["lon"].values[0] >= 0)
    assert(x["lon"].values[-1] <= 70)
    assert(x["lat"].values[0] >= -20)
    assert(x["lat"].values[-1] <= 20)
    if "time" in str(x.dims): assert False
    if len(x.values) == 0: assert False
            
def test_wrapper_mean_sst_2():
    x = wrapper_mean_sst(ds, ['1984-03-01','1984-03-01'])
    assert(x["lon"].values[0] >= 0)
    assert(x["lon"].values[-1] <= 360)
    assert(x["lat"].values[0] >= -90)
    assert(x["lat"].values[-1] <= 90)
    if "time" in str(x.dims): assert False
    if ("sst" in str(x.data_vars) == False): assert False
        
'''test function mean_sst'''
    
def test_mean_sst():
    x = mean_sst(ds, ['1984-03-01', '1984-03-04'], [0,-20,70,20])
    assert(x["lon"].values[0] >= 0)
    assert(x["lon"].values[-1] <= 70)
    assert(x["lat"].values[0] >= -20)
    assert(x["lat"].values[-1] <= 20)
    if "time" in str(x.dims): assert False
    if len(x.values) == 0: assert False
            
def test_mean_sst_2():
    x = mean_sst(ds, ['1984-03-01','1984-03-01'])
    assert(x["lon"].values[0] >= 0)
    assert(x["lon"].values[-1] <= 360)
    assert(x["lat"].values[0] >= -90)
    assert(x["lat"].values[-1] <= 90)
    if "time" in str(x.dims): assert False
    if ("sst" in str(x.data_vars) == False): assert False
    
'''test function exceptions_mean_sst'''

'''test that parameter bbox has right length'''
    
def test_BboxTooLong():
    with pytest.raises(BboxLengthError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'], [1,1,10,10,1])
            
def test_BboxTooShort():
    with pytest.raises(BboxLengthError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'], [1,1])
        
'''test that values within parameter bbox are right type'''
    
def test_BboxIsInt():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'], [1,1,10,'10'])
        
def test_BboxIsFloat():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'], ['1.65',1,10,10])
            
'''test that longitude within parameter bbox is inside bounds'''

def test_LongitudeOutsideBottomRange():
    with pytest.raises(LongitudeValueError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[-90,0,0,90])
            
def test_LongitudeOutsideTopRange():
    with pytest.raises(LongitudeValueError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[0,0,400,90])
            
'''test that latitude within parameter bbox is inside bounds'''

def test_LatitudeOutsideBottomRange():
    with pytest.raises(LatitudeValueError): 
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[0,-100,360,1])
            
def test_LatitudeOutsideTopRange():
    with pytest.raises(LatitudeValueError): 
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[0,-90,360,100])
            
def test_MinLatitudeGreaterThanMaxLatitude():
    with pytest.raises(LatitudeValueError): 
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[0,90,360,0])
            
'''test that latitude and longitude difference is big enough'''
    
def test_LongitudeCellsize():
    with pytest.raises(BboxCellsizeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[0,0,0.24,90])
        
def test_LongitudeCellsize_2():
    with pytest.raises(BboxCellsizeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[360,0,0,90])
    
def test_LatitudeCellsize():
    with pytest.raises(BboxCellsizeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-01'],[0,0,360,0.1])
            
'''test that parameter timeframe has right length'''

def test_TimeframeTooLong():
    with pytest.raises(TimeframeLengthError): 
        exceptions_mean_sst(ds, ['1984-03-01', '1984-03-01', '1984-03-01'])
            
def test_TimeframeTooShort():
    with pytest.raises(TimeframeLengthError): 
        exceptions_mean_sst(ds, ['1984-03-01'])
            
'''test that dates within parameter timeframe are inside bounds'''
                          
def test_TimeframeOutsideBottomRangeFirstP():
    with pytest.raises(TimeframeValueError): 
        exceptions_mean_sst(ds, ['1984-02-01','1984-03-01'])
            
def test_TimeframeOutsideTopRangeFirstP():
    with pytest.raises(TimeframeValueError): 
        exceptions_mean_sst(ds, ['1984-04-01','1984-03-01'])
            
def test_TimeframeOutsideBottomRangeSecondP():
    with pytest.raises(TimeframeValueError): 
        exceptions_mean_sst(ds, ['1984-03-01','1984-02-01'])
            
def test_TimeframeOutsideTopRangeSecondP():
    with pytest.raises(TimeframeValueError): 
        exceptions_mean_sst(ds, ['1984-03-01','1984-04-01'])
            
def test_StartDateBiggerThanEndDate():
    with pytest.raises(TimeframeValueError): 
        exceptions_mean_sst(ds, ['1984-03-04','1984-03-01'])
            
'''test that dates within parameter timeframe are real dates'''
                          
def test_InvalidDateFirstP():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-00','1984-03-02'])
            
def test_InvalidDateSecondP():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03-40'])
            
'''test that dates within parameter timeframe are valid datetimes'''
    
def test_InvalidDatetimeSyntaxFirstP_1():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984.03.01','1984-03-02'])
            
def test_InvalidDatetimeSyntaxSecondP_1():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984.03.02'])
            
def test_InvalidDatetimeSyntaxFirstP_2():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-0301','1984-03-02'])
            
def test_InvalidDatetimeSyntaxSecondP_2():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-0302'])
            
def test_InvalidDatetimeSyntaxFirstP_3():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['01-03-1984','1984-03-02'])
            
def test_InvalidDatetimeSyntaxSecondP_3():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-01','02-03-1984'])

def test_InvalidDatetimeSyntaxFirstP_4():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03','02-03-1984'])
        
def test_InvalidDatetimeSyntaxSecondP_4():
    with pytest.raises(ParameterTypeError):
        exceptions_mean_sst(ds, ['1984-03-01','1984-03'])

============================= test session starts =============================
platform win32 -- Python 3.8.6, pytest-6.2.1, py-1.10.0, pluggy-0.13.1
rootdir: C:\Users\DIGITA~1\AppData\Local\Temp\tmp_17wavpl
collected 35 items

_ipytesttmp.py ...................................                       [100%]

============================= 35 passed in 0.75s ==============================
